In [ ]:
import os
#TOKEN_FILE = "/run/secrets/kubernetes.io/serviceaccount/token" # not working in RHODS 1.25
TOKEN_FILE = "/mnt/secret_token/token"
with open(TOKEN_FILE) as f:
  token = f.read().strip()
route = os.environ["DSP_ROUTE"]

In [ ]:
import os
import kfp_tekton
from pipeline import my_pipeline

In [ ]:
cert="/run/secrets/kubernetes.io/serviceaccount/ca.crt"
ex = None
for i in range(5):
  try:
    print(f'try #{i}')
    client = kfp_tekton.TektonClient(host=route, existing_token=token, ssl_ca_cert=cert)
    print(f'try #{i} succeeded :)')
    break
  except Exception as e:
    import time;time.sleep(5)
    ex = e
else:
  raise Exception('Could not connect to TektonClient the after multiple tries :/') from ex

In [ ]:
from kfp_tekton.compiler import TektonCompiler
TektonCompiler().compile(my_pipeline, "results/pipeline.yaml")

In [ ]:
client.create_run_from_pipeline_func(pipeline_func=my_pipeline, arguments={})